In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 사전작업: Colab 환경 설정

In [ ]:
!pwd

/content


In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
### Torch 재설치 후 재시작 필수 ###

In [ ]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302


In [ ]:
# for transformers, 최신버전은 에러발생
# !pip install transformers==4.28.1
!pip install transformers
!pip install accelerate
# !pip install accelerate -U

!pip install colossalai==0.2.7
# !pip install colossalai
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1
!pip install datasets
!pip install jsonlines
!pip install loralib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.7/686.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.0/396.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.0
    Uninstalling pydantic-2.2.0:
      Successfully uninstalled pydantic-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatibl

In [ ]:
%cd /content/drive/MyDrive/
!git clone https://github.com/oglee815/mygpt-lecture.git
%cd /content/drive/MyDrive/mygpt-lecture

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 249 (delta 67), reused 97 (delta 51), pack-reused 129
Receiving objects: 100% (249/249), 37.35 MiB | 12.50 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [ ]:
# import
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline, PreTrainedTokenizerFast
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/mygpt-lecture")
sys.path.append("/content/drive/MyDrive/mygpt-lecture/code/")

In [ ]:
from utils import SFT_dataset, DataCollatorForSupervisedDataset
from typing import Optional, Dict, Sequence
import jsonlines

### KoGPT2 테스트

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>',
                                                    padding_side="right", model_max_length=512)

model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2').to('cuda')

print(tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o"))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


In [ ]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = tokenizer.pad_token #"[PAD]"
DEFAULT_EOS_TOKEN = tokenizer.eos_token # "</s>"
DEFAULT_BOS_TOKEN = tokenizer.bos_token #"</s>"
DEFAULT_UNK_TOKEN = tokenizer.unk_token # "</s>"

In [ ]:
## 추론 테스트
generator_skt_gpt = pipeline('text-generation', model=model, tokenizer=tokenizer, device=model.device)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id, # \n
    max_new_tokens=256,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

list_prompt = [ '동해물과 백두산이', '불고기 요리 만드는 방법 알려줘']

print(list_prompt)
list_result = generator_skt_gpt(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print(('#'*70))
    print(('completion:\n%s'%(result[0]['generated_text'])))

['동해물과 백두산이', '불고기 요리 만드는 방법 알려줘']
######################################################################
completion:
동해물과 백두산이 만나는 지점인 강원 고성군 통일전망대~통일전망대 구간을 잇는 총 연장 2.2km(왕복 4차로) 도로 개설공사를 올해 말 착공할 계획이다.
총 사업비는 860억원이다.
국토부는 남북협력기금과 민간자본을 투입해 2018년까지 완공할 계획이다.
이 도로가 개통되면 서울~속초 간 동서고속화철도(KTX) 운행 횟수가 기존 2시간에서 3시간으로 10분가량 줄어들 것으로 국토부는 내다봤다.
동서고속화철도는 강원도 속초시 중앙동~삼척시 동명동~양양군 서면~고성군 토성면~평창군 진부면~강릉시 주문진읍~속초시 노학동~정선군 남면~홍천군 북방면~춘천시 신북읍~양구군 화촌면~원주시 단구동~횡성군 횡성읍~영월군 소흘읍~옥천군 서석면~화천군 사내면~괴산군 청천면~증평군 도안면~음성군 비봉면~보은군 도암면~충주시 가금면~음성군 문의면~단양군 영춘면~제천시 봉양읍~제천시 봉양읍~청
######################################################################
completion:
불고기 요리 만드는 방법 알려줘요. 네. 자~ 요거 보시면 돼요? 네. 이게 뭔지 아시죠? 네. 예. 네. 네. 근데 저는 이제 육즙이 굉장히 좋은 거 같애요. 네. 어~ 그니까 육즙이 아주 좋죠. 네. 아~ 그래요? 음~ 그래서 육즙이 너무 좋아요. 네. 그럼 육즙에 대해서 좀 알아보고 싶으신가 봐요. 네. 육즙이 정말 중요하잖아요. 네. 그렇죠. 맞아요. 네. 그러면 육즙은 어떤 건가요? 육즙이 뭐냐면 우리가 흔히 먹는 보통 육즙이라고 할 수 있는 채소들이 전부 다 이렇게 되는 거예요. 네. 네네. 네. 그런데 이 채소들 중에 어느 것 하나 가장 맛있을 수가 없거든요. 네. 그러니까 네. 한 번 먹어보시면 좋을 것 같아요. 네. 맞어요.

### General Instruction Model 테스트

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunjae/skt-kogpt2-kullm-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>',
                                                    padding_side="right", model_max_length=512)

model = AutoModelForCausalLM.from_pretrained('hyunjae/skt-kogpt2-kullm-v2').to('cuda')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
## 추론 테스트
generator_general_model = pipeline('text-generation', model=model, tokenizer=tokenizer, device=model.device)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id, # \n
    max_new_tokens=256,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

list_prompt = [ '동해물과 백두산이', '불고기 요리 만드는 방법 알려줘']

PROMPT= "### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:{instruction}\n### 응답:"
prompt_list = [ PROMPT.format_map({'instruction':x}) for x in list_prompt]

list_result = generator_general_model(prompt_list, **generation_args)
for prompt, result in zip(prompt_list, list_result):
    print(('#'*70))
    print(('completion:\n%s'%(result[0]['generated_text'])))

######################################################################
completion:
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:동해물과 백두산이
### 응답:물론이죠! 다음은 두 산을 모두 포함하는 업데이트된 표입니다:| 산의 이름 | 설명 || --- | --s- || 그랜드 캐년 | 지구상에서 가장 높은 봉우리 | 알래스카, 그린란드, 캐나다, 에콰도르 || 퀸즈랜드 | 호주 | 노바스코샤, 퀸즈, 뉴사우스웨일스주 || 그레이트 배리어 리프, 미국 | 콜로라도, 유타 || 스노우 마운틴 | 미국 | 노스 캐롤라이나 주 | 애리조나, 뉴멕시코 || 샌디에이고 | 캘리포니아 | 로스앤젤레스, 텍사스 || 산호세 | 멕시코 | 산타크루즈, 하와이 || 오클랜드 | 애리조나 | 로스 앤젤레스 || 엘리스 아일랜드 | 플로리다 | 사우스 다코타 주, 플로리다, 미시시피 || 아이다호 | 아이다호, 캘리포니아 | 아이다
| 오리건 | 텍사스 | 텍사스, 텍사스 |
######################################################################
completion:
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:불고기 요리 만드는 방법 알려줘
### 응답:불고기 요리는 간단하고 맛있는 음식입니다. 다음은 불고기 요리를 만드는 방법에 대한 단계별 가이드입니다:

재료
- 불고기 1파운드
- 다진 마늘 2쪽
- 잘게 썬 모짜렐라 치즈 1컵
- 올리브 오일 2 큰술
- 맛볼 소금과 후추
- 서빙용 신선한 파슬리

만드는 방법
1. 오븐을 375°F(190°C)로 예열합니다.
2. 큰 냄비나 더치 오븐에 올리브 오일을 두르고 중간 불에 올립니다. 양파와 마늘을 넣고 부드러워질 때까지 약 5분간 조리합니다.
3. 3. 불고기를 냄비에 넣고 끓입니다.
4. 불을 약하게 줄이고 뚜껑을 덮은 후 25

### Stage 1-1 Domain Adaptive Pre-training

In [ ]:
train_dataset = SFT_dataset(data_path='/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/domain_adaptive_kuksundo_pretrain.jsonl', tokenizer=tokenizer)
eval_dataset  = None  # eval은 안함
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:다음 주제에 대하여, 위키피디아에 나와 있는 그대로 말해주세요.
주제:국선도의 개요1

### 응답:
국선도(밝돌법)는 9,800년의 역사와 전통을 가진 것으로 알려진 한국 고유의 심신 수련법으로, 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 자기계발 수련법이다.    9단법 37단계 430여 개 동작으로 구성된 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 한국 고유의 명상과학 프로그램이다.    심신수련과 인재 양성의 도법으로 약 9,800년 전부터 전해져 내려와 고조선의 소도, 고구려의 조의선인제도, 신라의 화랑도 등으로 불렸고, 고려 이후에는 산중 도법으로 전해 내려오다가 산중 수도인 청산(靑山, 본명 고한영)이 스승 청운도인의 "모두의 유익을 위해 도법을 전하라"는 명을 받고 1967년에 사회로 나와 국선도(밝돌법)를 현대 인류 사회에 다시 소개하였다.    밝돌법은 국선도의 본래 이름으로, ‘밝’은 생명의 에너지(태양, 밝음), ‘돌’은 순환하는(돌고 도는) 우주 자연의 법칙을 뜻하며, ‘밝돌법’은 우주 질서의 순리에 맞게 생명을 양생(養生)하는 수련방법을 의미한다.    국선도는 상고시대부터 전해오는 한국의 고유한 전통 심신수련법이다.</s>
Tokenizing inputs... This may take some time...


In [ ]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_dataset[0]['input_ids']))

'### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:다음 주제에 대하여, 위키피디아에 나와 있는 그대로 말해주세요.\n주제:국선도의 개요1\n\n### 응답:국선도(밝돌법)는 9,800년의 역사와 전통을 가진 것으로 알려진 한국 고유의 심신 수련법으로, 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 자기계발 수련법이다.    9단법 37단계 430여 개 동작으로 구성된 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 한국 고유의 명상과학 프로그램이다.    심신수련과 인재 양성의 도법으로 약 9,800년 전부터 전해져 내려와 고조선의 소도, 고구려의 조의선인제도, 신라의 화랑도 등으로 불렸고, 고려 이후에는 산중 도법으로 전해 내려오다가 산중 수도인 청산(靑山, 본명 고한영)이 스승 청운도인의 "모두의 유익을 위해 도법을 전하라"는 명을 받고 1967년에 사회로 나와 국선도(밝돌법)를 현대 인류 사회에 다시 소개하였다.    밝돌법은 국선도의 본래 이름으로, ‘밝’은 생명의 에너지(태양, 밝음), ‘돌’은 순환하는(돌고 도는) 우주 자연의 법칙을 뜻하며, ‘밝돌법’은 우주 질서의 순리에 맞게 생명을 양생(養生)하는 수련방법을 의미한다.    국선도는 상고시대부터 전해오는 한국의 고유한 전통 심신수련법이다.</s>'

In [ ]:
## 학습 (10min)
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mygpt-lecture/domain_adaptive_model", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs= 30, # number of training epochs
    gradient_accumulation_steps= 1,
    learning_rate=3e-5,
    per_device_train_batch_size= 6, # batch size for training
    per_device_eval_batch_size= 6,  # batch size for evaluation
    eval_steps = 10, # Number of update steps between two evaluations.
    logging_steps=1,
    prediction_loss_only=True, #?
    fp16=True
    # bf16=True,
    # tf32=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# trainer.train()
# trainer.save_state()
# safe_save_model_for_hf_trainer(trainer=trainer, output_dir="./domain_adaptive")

In [ ]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/mygpt-lecture/domain_adaptive_model")
# trainer.save_state()
# safe_save_model_for_hf_trainer(trainer=trainer, output_dir="/content/drive/MyDrive/KoChatGPT/general_model")

Step,Training Loss
1,5.832500
2,5.641700
3,4.990900
4,5.450000
5,5.252700
6,5.005700
7,4.534900
8,4.534100
9,4.626100
10,4.329800


#### General Model vs Domain Adaptive Pretraining Model 추론 테스트

In [ ]:
## 추론 테스트
generator_general = pipeline('text-generation', model="hyunjae/skt-kogpt2-kullm-v2", tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id, # \n
    max_new_tokens=128,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

In [ ]:
# 제너럴 모델은 국선도에 대해서 잘 알까?

list_prompt = ['불고기 요리 만드는 방법 알려줘', '국선도란 무엇입니까?']
list_prompt = [PROMPT.format_map({'instruction' : tmp}) for tmp in list_prompt]

print(list_prompt)
list_result = generator_general(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print(('#'*70))
    print(('completion: %s'%(result[0]['generated_text'])))

['### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:불고기 요리 만드는 방법 알려줘\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도란 무엇입니까?\n### 응답:']
######################################################################
completion: ### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:불고기 요리 만드는 방법 알려줘
### 응답:불고기 요리를 만드는 간단한 레시피를 소개합니다:

재료
- 불고기 1파운드
- 다진 큰 양파 1개
- 다진 마늘 2쪽
- 말린 오레가노 1작은술
- 맛볼 소금과 후추
- 물기를 빼고 헹군 검은콩 1캔(14.5온스)
- 깍둑 썰기 한 붉은 피망 1개
- 간 파마산 치즈 1컵
- 올리브 오일 1 큰술
- 다진 신선한 고수 1/4 컵
- 잘게 썬 모짜렐라 치즈 1/2 컵
- 장식용 다진 토마토 1캔(15온스)

만드는 방법
1. 큰
######################################################################
completion: ### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도란 무엇입니까?
### 응답:국선은 대륙과 해양이 서로 연결되어 있는 두 국가 간의 무역 및 경제 관계를 의미합니다. 즉, 한 국가가 다른 국가와의 무역을 통해 이익을 얻을 수 있다는 것을 의미합니다. 국선은 국제 무역과 경제에서 중요한 역할을 합니다.


In [ ]:
## 추론 테스트, domain_adaptive_model
generator_general_domain_adaptive = pipeline('text-generation', model="/content/drive/MyDrive/mygpt-lecture/domain_adaptive_model", tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id, # \n
    max_new_tokens=128,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

In [ ]:
# 도메인 어뎁티브 프리트레이닝 모델은 국선도에 대해서 얼마나 알고 있을까?

list_prompt =['불고기 요리 만드는 방법 알려줘', '국선도란 무엇입니까?']
list_prompt = [PROMPT.format_map({'instruction' : tmp}) for tmp in list_prompt]

print(list_prompt)
list_result = generator_general_domain_adaptive(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print(('#'*70))
    print(('completion: %s'%(result[0]['generated_text'])))

['### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:불고기 요리 만드는 방법 알려줘\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도란 무엇입니까?\n### 응답:']
######################################################################
completion: ### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:불고기 요리 만드는 방법 알려줘
### 응답:불고기 요리는 다음과 같은 방법을 따라해보자:
1. 불고기 덩어리를 깨끗이 씻어서 따로 보관합니다.
2. 큰 프라이팬에 올리브 오일을 두르고 중간 불에 올립니다.
3. 3. 불고기에 소금과 후추로 간을 합니다.
4. 4. 불고기가 완전히 가열될 때까지 볶습니다.
5. 5. 불고기를 넣고 저어주고 소스가 잘 섞이도록 버무립니다.
6. 6. 불고기 위에 붓고 맛있게 드세요!
######################################################################
completion: ### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도란 무엇입니까?
### 응답:국선도(밝돌법)는 9,800년의 역사와 전통을 가진 것으로 알려진 한국 고유의 심신 수련법으로, 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 자기계발 수련법이다.


### Stage 1-2 SFT 훈련

In [ ]:
model = AutoModelForCausalLM.from_pretrained("hyunjae/skt-kogpt2-kullm-v2").to('cuda')

In [ ]:
train_dataset = SFT_dataset(data_path='/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/kuksundo_instruction_train.jsonl', tokenizer=tokenizer)
eval_dataset  = None  # eval은 안함
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도는 얼마나 오래된 전통을 가진 심신수련법인가요?
### 응답:
국선도는 9,800년의 역사와 전통을 가진 것으로 알려져 있습니다.</s>
Tokenizing inputs... This may take some time...


In [ ]:
## 학습 (10min)
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mygpt-lecture/domain_instructin_tuning", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs= 30, # number of training epochs
    gradient_accumulation_steps= 1,
    learning_rate=1e-5,
    per_device_train_batch_size= 6, # batch size for training
    per_device_eval_batch_size= 6,  # batch size for evaluation
    eval_steps = 10, # Number of update steps between two evaluations.
    logging_steps=1,
    prediction_loss_only=True, #?
    fp16=True
    # bf16=True,
    # tf32=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/mygpt-lecture/domain_instructin_tuning")

In [ ]:
## 추론 테스트
generator_instruction_tuning = pipeline('text-generation', model="/content/drive/MyDrive/mygpt-lecture/domain_instructin_tuning", tokenizer=tokenizer)
# generator = pipeline('text-generation', model=model.cpu(), tokenizer=tokenizer, config={'max_length':800})

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id, # \n
    max_new_tokens=256,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

In [ ]:
list_test = open("/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/kuksundo_instruction_test.jsonl", 'r')
test_data = [json.loads(x) for x in list_test]

In [ ]:
test_data[1]

{'instruction': '국선도를 통해 누구든지 건강한 신체와 마음, 정신을 가질 수 있지만, 그 외에 어떤 장점이 있나요?',
 'output': '국선도는 자기계발 수련법으로, 심신 건강 뿐만 아니라 인격성장, 도덕성 향상, 스스로의 능력 향상 등 다양한 면에서 이점을 제공합니다.'}

In [ ]:
list_prompt = [ x['instruction'] for x in test_data]
list_prompt = [PROMPT.format_map({'instruction' : tmp}) for tmp in list_prompt]
chatgpt_answer = [ x['output'] for x in test_data]

print(list_prompt)
list_result = generator_instruction_tuning(list_prompt, **generation_args)
for prompt, result, answer in zip(list_prompt, list_result, chatgpt_answer):
    print(('#'*70))
    print(prompt)
    print(('Model output: %s'%(result[0]['generated_text'].replace(prompt,""))))
    print("ChatGPT output:", answer)

['### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도가 이전에 불린 다른 이름은 무엇인가요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도를 통해 누구든지 건강한 신체와 마음, 정신을 가질 수 있지만, 그 외에 어떤 장점이 있나요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도 수련을 통해 어떤 자세를 터득할 수 있는가요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도의 역사는 어떻게 시작되었나요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도의 수련 방법은 어떤 것이 있나요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도에서 어떤 단계에서 천지인 합일 경지에 진입하게 되는가요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도 수련에서의 단전행공이란 무엇인가요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도 수련 중 적절한 호흡법은 무엇인가요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도 밝돌법 수련이 정신적인 면에서 어떤 효과를 줄까요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도 밝돌법 수련이 피부에 어떤 효과를 줄까요?\n### 응답:', '### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:국선도에서 자연치유란 무엇을 의미하는가?\n### 응답:', '### system:사용자의 질문에 맞는 적

### Stage 2. Rating Model 만들기

In [ ]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/KoChatGPT")
sys.path.append("/content/drive/MyDrive/KoChatGPT/code/")

In [ ]:
# import
import argparse

# import loralib as lora
import torch
torch.cuda.empty_cache()
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from datasets import load_dataset
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

import os
import json


from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunjae/skt-kogpt2-kullm-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>',
                                                    padding_side="right", model_max_length=512)

# data config
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = tokenizer.pad_token # "[PAD]"
DEFAULT_EOS_TOKEN = tokenizer.eos_token
DEFAULT_BOS_TOKEN = tokenizer.bos_token
DEFAULT_UNK_TOKEN = tokenizer.unk_token

PROMPT= "### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:{instruction}\n### 응답:{output}"

/usr/local/lib/python3.10/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at /dev/null:241 (Triggered internally at ../aten/src/ATen/core/dispatch/OperatorEntry.cpp:150.)
  self.m.impl(name, dispatch_key, fn)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
from chatgpt.models.gpt.gpt_rm import GPTRM
model = GPTRM(pretrained="hyunjae/skt-kogpt2-kullm-v2").to('cuda')

In [ ]:
train_rm=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/kuksundo_train_rm.jsonl", 'r'):
    train_rm.append(json.loads(a))

test_rm=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/kuksundo_test_rm.jsonl", 'r'):
    test_rm.append(json.loads(a))

In [ ]:
train_rm[-1]

{'prompt': '국선도의 기화법을 배우면 어떤 장점이 있을까요?',
 'rejected': '국선도의 기화법을 배우면 체력과 근력을 향상시키고, 자기 통제 능력과 집중력을 향상시킵니다. 또한, 일상 생활에서의 자세와 태도 개선, 정서 조절 등으로 전반적인 건강과 안녕에 도움이 될 수 있습니다.',
 'chosen': '국선도의 기화법을 배우면 체력과 기술적인 능력을 향상시킬 수 있습니다.'}

In [ ]:
print("Train set size:", len(train_rm))
print("Test set size:", len(test_rm))

Train set size: 323
Test set size: 27


In [ ]:
len(test_rm)

27

In [ ]:
train = [{'prompt':"### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:"+a['prompt']+"\n### 응답:", 'chosen':a['chosen'], 'rejected':a['rejected']} for a in train_rm]
test = [{'prompt':"### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.\n### 사용자:"+a['prompt']+"\n### 응답:", 'chosen':a['chosen'], 'rejected':a['rejected']} for a in test_rm]
train_dataset = RewardDataset(train, tokenizer, 256)
eval_dataset = RewardDataset(test, tokenizer, 256)

100%|██████████| 27/27 [00:00<00:00, 1397.51it/s]


In [ ]:
rm_output_path="/content/drive/MyDrive/mygpt-lecture/output_2_RM_kuksundo/"
if not os.path.exists(rm_output_path):
    os.makedirs(rm_output_path)

In [ ]:
strategy = NaiveStrategy()

In [ ]:
optim = Adam(model.parameters(), lr=1e-4)

In [ ]:
# batch_size here is expected to be C(k,2), k means # response of each prompt
# be limited with the format of dataset 'Dahoas/rm-static', we'd better use batch_size as 1
trainer = RewardModelTrainer(model=model,
                             strategy=strategy,
                             optim=optim,
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=8,
                             max_epochs=3)

In [ ]:
# train!!
trainer.fit(use_lora=0)

Train step of epoch 0: 100%|██████████| 41/41 [00:33<00:00,  1.24it/s, loss=0.132, dist_mean=7.45]

Train step of epoch 1: 100%|██████████| 41/41 [00:33<00:00,  1.23it/s, loss=0.0411, dist_mean=15.8]

Train epoch: 100%|██████████| 3/3 [01:38<00:00, 32.90s/it]


In [ ]:
torch.save(model.state_dict(), rm_output_path + "/pytorch_model.bin")
model.model.save_pretrained(rm_output_path)  # config.json 생성

In [ ]:
# 보상모델 체크
def inference_RM(input_text=None):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]
    return output_reward

In [ ]:
# Test 데이터셋 에 대해 Accuracy 측정해보기
correct = 0
for a in test:
    input_text = a['prompt'] + a['chosen']
    print("--------------")
    print("@ 좋은 답변")
    print(input_text,output_reward_good)
    output_reward_good = inference_RM(input_text=input_text)

    input_text =a['prompt'] + a['rejected']

    output_reward_bad = inference_RM(input_text=input_text)
    print("@ 나쁜 답변")
    print(input_text, output_reward_bad)

    if output_reward_good > output_reward_bad:
        correct +=1

--------------
@ 좋은 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도는 어떤 도법으로 알려져 있나요?
### 응답:국선도는 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 알려져 있습니다. -0.2818417
@ 나쁜 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도는 어떤 도법으로 알려져 있나요?
### 응답:국선도는 주로 기본적인 군용 기술인 무예로 알려져 있습니다. -1.3789477
--------------
@ 좋은 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도에는 몇 개의 몸 동작과 호흡법이 체계적으로 구성되어 있나요?
### 응답:국선도에는 수백 개의 몸 동작과 정교한 호흡법이 체계적으로 구성되어 있습니다. -0.17336902
@ 나쁜 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도에는 몇 개의 몸 동작과 호흡법이 체계적으로 구성되어 있나요?
### 응답:국선도에는 8가지 몸 동작과 호흡법이 체계적으로 구성되어 있습니다. -0.03607969
--------------
@ 좋은 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도의 맥을 이어온 곳은 어디인가요?
### 응답:국선도의 맥을 이어온 곳은 고구려의 조의선인제도, 백제의 수사제도, 신라의 화랑제도였습니다. 0.5941783
@ 나쁜 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세요.
### 사용자:국선도의 맥을 이어온 곳은 어디인가요?
### 응답:중국 도쿄대학에서 시작되었지만, 국선도는 현재 국제적으로 널리 전파되어 있으며, 전 세계 다양한 국가와 지역에서 수련과 교육 활동이 이어지고 있습니다. -3.1562598
--------------
@ 좋은 답변
### system:사용자의 질문에 맞는 적절한 응답을 생성하세

In [ ]:
correct/len(test)

0.9259259259259259

### Stage 3. PPO 학습

In [ ]:
# import
import argparse
from copy import deepcopy

import sys
sys.path.append("/content/drive/MyDrive/mygpt-lecture")
sys.path.append("/content/drive/MyDrive/mygpt-lecture/code/")


import pandas as pd
import torch
torch.cuda.empty_cache()
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMActor, BLOOMCritic
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.opt import OPTActor, OPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam
from utils import SFT_dataset, DataCollatorForSupervisedDataset
from typing import Optional, Dict, Sequence
import jsonlines

## wy 추가
import json
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# ## clossalAI error 해결
# os.environ['RANK'] = '0'
# os.environ['LOCAL_RANK'] = '-1'
# os.environ['WORLD_SIZE'] = '1'
# os.environ['MASTER_ADDR'] = '127.0.0.1'
# os.environ['MASTER_PORT'] = '42043'

/usr/local/lib/python3.10/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at /dev/null:241 (Triggered internally at ../aten/src/ATen/core/dispatch/OperatorEntry.cpp:150.)
  self.m.impl(name, dispatch_key, fn)


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("hyunjae/skt-kogpt2-kullm-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>',
                                                    padding_side="right", model_max_length=512)
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = tokenizer.pad_token # "[PAD]"
DEFAULT_EOS_TOKEN = tokenizer.eos_token
DEFAULT_BOS_TOKEN = tokenizer.bos_token
DEFAULT_UNK_TOKEN = tokenizer.unk_token

PROMPT= "### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.\n### 사용자:{instruction}\n### 응답:"

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--train_data_path_ppo', type=str, default='/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/kuksundo_train_ppo.jsonl')
parser.add_argument('--test_data_path_ppo', type=str, default='/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/kuksundo_test_ppo.jsonl')
parser.add_argument('--output_dir', type=str, default='/content/drive/MyDrive/mygpt-lecture/output_3_PPO')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--num_episodes', type=int, default=10)
parser.add_argument('--max_timesteps', type=int, default=3)
parser.add_argument('--update_timesteps', type=int, default=3)
parser.add_argument('--max_epochs', type=int, default=5)
parser.add_argument('--train_batch_size', type=int, default=6)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_length', type=int, default=256)
parser.add_argument('--lr', type=float, default=1e-6)
args = parser.parse_args(args=[])

# for test
# args.output_dir = '/context/drive/MyDrive/KoChatGPT/output_3_PPO'
# args.pretrain = 'skt/kogpt2-base-v2'  # pretrained 모델 가져오기

## 이곳 수정!!
args.pretrain_actor = '/content/drive/MyDrive/mygpt-lecture/domain_instruction_tuning/'  # SFT 모델 가져오기
args.pretrain_critic = '/content/drive/MyDrive/mygpt-lecture/output_2_RM_kuksundo'  # RM 모델 가져오기
# args.pretrain_actor = args.pretrain
# args.pretrain_critic = args.pretrain

args.max_epochs=2
args.num_episodes=3

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

Namespace(train_data_path_ppo='/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_train_ppo.jsonl', test_data_path_ppo='/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_test_ppo.jsonl', output_dir='/content/drive/MyDrive/KoChatGPT/output_3_PPO', model='gpt2', pretrain=None, num_episodes=3, max_timesteps=3, update_timesteps=3, max_epochs=2, train_batch_size=6, lora_rank=0, max_length=256, lr=1e-06, pretrain_actor='/content/drive/MyDrive/KoChatGPT/domain_instruction_tuning/', pretrain_critic='/content/drive/MyDrive/KoChatGPT/output_2_RM_kuksundo')


In [ ]:
# configure strategy
strategy = NaiveStrategy()

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    actor = GPTActor(pretrained=args.pretrain_actor).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=args.pretrain_critic).to(torch.cuda.current_device())
    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [ ]:
# configure optimizer
actor_optim = Adam(actor.parameters(), lr=args.lr)
critic_optim = Adam(critic.parameters(), lr=args.lr)

In [ ]:
# setting the models
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = strategy.prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [ ]:
train_ppo=[]
for a in open(args.train_data_path_ppo, 'r'):
    train_ppo.append(PROMPT.format_map({'instruction':json.loads(a)['prompt']}))

test_ppo=[]
for a in open(args.test_data_path_ppo, 'r'):
    test_ppo.append(PROMPT.format_map({'instruction':json.loads(a)['prompt']}))

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=args.max_length, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [ ]:
train_ppo[0]

In [ ]:
# configure trainer
trainer = PPOTrainer(strategy,
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=args.max_epochs,
                     train_batch_size=6,
                     tokenizer=tokenize_fn,
                     max_length=args.max_length,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

## train!
trainer.fit(train_ppo,  # 입력 prompt
            num_episodes=args.num_episodes,
            max_timesteps=args.max_timesteps,
            update_timesteps=args.update_timesteps)

# ## save
# # save model checkpoint after fitting on only rank0
# strategy.save_model(actor, os.path.join(args.output_dir, 'actor.pt'), only_rank0=True)
# # save optimizer checkpoint on all ranks
# strategy.save_optimizer(actor_optim,
#                         os.path.join(args.output_dir, 'actor_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
#                         only_rank0=False)

Train epoch [1/2]: 100%|██████████| 4/4 [00:03<00:00,  1.31it/s, actor_loss=0, critic_loss=0.00943]

Train epoch [1/2]: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s, actor_loss=0.232, critic_loss=0.00319]

Train epoch [1/2]: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s, actor_loss=0.0346, critic_loss=0.0114]

Episode [3/3]: 100%|██████████| 3/3 [01:08<00:00, 22.86s/it]


In [ ]:
torch.save(actor.state_dict(), args.output_dir + "/pytorch_model.bin")
actor.model.save_pretrained(args.output_dir)  # config.json 생성

#### SFT vs PPO Inference 비교

In [ ]:
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id, # \n
    max_new_tokens=256,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

In [ ]:
# from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained(args.pretrain_actor).to('cuda')
from transformers import pipeline
## 추론 테스트
generator_ppo = pipeline('text-generation', model=args.output_dir , tokenizer=tokenizer)
list_result_ppo = generator_ppo(test_ppo, **generation_args)

generator_sft = pipeline('text-generation', model=args.pretrain_actor , tokenizer=tokenizer)
list_result_sft = generator_sft(test_ppo, **generation_args)

In [ ]:
for prompt, ppo_result, sft_result in zip(test_ppo, list_result_ppo, list_result_sft):
    print(('#'*70))
    print(prompt)
    print("PPO: ",ppo_result[0]['generated_text'].replace(prompt,""))
    print("SFT: ",sft_result[0]['generated_text'].replace(prompt,""))

######################################################################
### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.
### 사용자:국선도는 어떤 사람들에게 추천되는 수련법인가요?
### 응답:
PPO:  국선도의 수련법은 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 수련법입니다.
SFT:  국선도의 수련법은 군인과 경찰 등의 현장에서 근무하는 사람들에게 추천됩니다.
######################################################################
### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.
### 사용자:밝돌법이라는 이름의 유래는 무엇인가요?
### 응답:
PPO:  도인도송 내용에서 밝돌법은 가운데로 숨쉬는 것을 의미합니다.
SFT:  도인도송 내용에서 밝돌법은 천기와 지기가 돌단자리에 모이고 힘이 반시계 방향으로 몸을 돌리며 발과 항문에 은은한 힘을 줍니다.
######################################################################
### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.
### 사용자:국선도를 통해 건강한 몸을 유지하는 방법은 무엇인가요?
### 응답:
PPO:  국선도의 건강한 몸가짐을 유지하려면 꾸준한 수행이 필요합니다.
SFT:  국선도의 건강한 몸가짐을 유지하려면 꾸준한 수행이 필요합니다.
######################################################################
### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적

#### Pipeline 없이 Inference 하는 법

In [ ]:
## inference
def generation(model, input_text):
    input_ids = tokenizer(input_text, return_tensors='pt').to(
        torch.cuda.current_device())

    outputs = model.generate(**input_ids,
                             num_beams=4,
                            repetition_penalty=2.0,
                            no_repeat_ngram_size=4,
                            eos_token_id=tokenizer.eos_token_id,
                            max_new_tokens=256,
                            do_sample=True,
                            top_k=50,
                            early_stopping=True
                             )

    # print(outputs)
    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print('#' * 70)
    print(output)
    return output

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(args.output_dir).to('cuda').eval()

for input_text in test_ppo:
    output = generation(model, input_text)

######################################################################
['### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.\n### 사용자:국선도는 어떤 사람들에게 추천되는 수련법인가요?\n### 응답:국선도(밝돌법)는 사람들이 수련하여 건강해질 수 있는 수련법이다.']
######################################################################
['### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.\n### 사용자:밝돌법이라는 이름의 유래는 무엇인가요?\n### 응답:도인도송 내용에서 국선도법(밝돌법)은 인도송 내용에서 유래했습니다.']
######################################################################
['### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.\n### 사용자:국선도를 통해 건강한 몸을 유지하는 방법은 무엇인가요?\n### 응답:국선도에서 건강한 몸을 유지하는 방법에는 여러 가지가 있습니다.']
######################################################################
['### system:당신은 국선도 전문가입니다. 국선도에 대한 사용자의 질문에 대해 적절한 응답을 생성하세요.\n### 사용자:국선도를 수행하는 데 필요한 시간과 노력은 어느 정도인가요?\n### 응답:국선도의 수행을 위해서는 중기단법(中氣丹法), 건곤단법(乾坤丹法), 진공단법(眞空丹法)이 필요합니다.']
######################################################################
['### system:당신은 국선도 전문가입니다. 국선

In [ ]:
!rm -rf /content/drive/MyDrive/KoChatGPT/temp/